### Detect the faces from the webcam and apply PCA on it

In [142]:
import cv2
import numpy as np
import os

In [143]:
def PCA(reference_faces_vector):
    """
    Creating the PCA model
    
    ## Parameters 
    face_vector : contain the training images where each column contain one image of size (N^2 * 1)
    
    ## Returns
    weights : vector contain the similarity between each image and eigen vectors

    avg_face_vector : the mean image, used in testing

    eigen_faces : contain eigen vectors, used in testing
    """
    #get the mean image
    avg_face_vector = reference_faces_vector.mean(axis=1)
    avg_face_vector = avg_face_vector.reshape(reference_faces_vector.shape[0], 1)
    #subtract the mean image from the images
    normalized_face_vector = reference_faces_vector - avg_face_vector

    #calculate covariance matrix
    covariance_matrix = np.cov(np.transpose(normalized_face_vector)) 

    #get eigen values and eigen vectors
    eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)

    #select best k eigen vectors . this variable is changable according to the data set
    k = 30
    index_of_max_k_eigen_values = np.argpartition(eigen_values, -k)[-k:]
    k_eigen_vectors = []

    for i in range(len(index_of_max_k_eigen_values)):
        k_eigen_vectors.append(eigen_vectors[index_of_max_k_eigen_values[i]])

    k_eigen_vectors = np.asarray(k_eigen_vectors)

    eigen_faces = k_eigen_vectors.dot(normalized_face_vector.T)
    weights = (normalized_face_vector.T).dot(eigen_faces.T)

    return weights, avg_face_vector, eigen_faces


In [144]:
def test(test_img, weights, avg_face_vector, eigen_faces):
    """
    Recognize faces in test image

    ## Parameters
    test_img : image to recognize faces in , it is in gray scale

    weights : the weights resulting from PCA model

    ave_face_vector : average face from the training images

    eigen_faces : eigen vector resulting from PCA model
    
    ## Returns
    index : index of the matched image

    """
    test_img = test_img.reshape(test_img.shape[0] * test_img.shape[1], 1)
    test_normalized_face_vector = test_img - avg_face_vector
    test_weight = (test_normalized_face_vector.T).dot(eigen_faces.T)
    index = np.argmin(np.linalg.norm(test_weight - weights, axis=1))
    return index

In [145]:
# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml")

# Set up a list of reference faces
directory = 'images'
reference_faces = []
reference_labels = []

# Access the directory that contains all the images folders
for foldername in os.listdir(directory):
    # Access all the images files in each folder
       for filename in os.listdir(directory + '/' + foldername):
            # Read each face image and store it and its label in the lists
            face_image = cv2.imread(directory + '/' + foldername + '/' + filename, cv2.IMREAD_GRAYSCALE)
            reference_faces.append(face_image)
            reference_labels.append(foldername)

# Convert the lists to arrays
reference_faces = np.asarray(reference_faces)
reference_labels = np.asarray(reference_labels)

reference_faces_vector = []

for face in reference_faces:
    reference_faces_vector.append(face.flatten())
reference_faces_vector = np.asarray(reference_faces_vector).transpose()


# Get the weights, eigen vectors and eigen values of the vector
weights, avg_face_vector, eigen_faces = PCA(reference_faces_vector)

In [146]:
# To capture video from webcam
camera = cv2.VideoCapture(0)

while True:
    # Read the frame
    stream, frame = camera.read()

    # Calculate the PCA features of the frame

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Draw a bounding box around the detected face and label it with the closest reference face
    faces_detected = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    if len(faces_detected) > 0:
        # Initialize an empty list to store the cropped face images

        for (x, y, w, h) in faces_detected:
            face_cropped_img = gray_frame[y:y+h, x:x+w]
            resized_face_cropped_img = cv2.resize(face_cropped_img,(250,250))
            closest_idx = test(resized_face_cropped_img, weights, avg_face_vector, eigen_faces)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, f"{reference_labels[closest_idx]}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Exit if the key "q" is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
camera.release()
cv2.destroyAllWindows()